# 🔥 Fine-Tuning AI - التدريب السحابي المتقدم
## نظام تدريب نماذج الذكاء الاصطناعي باستخدام Google Colab

### المميزات:
- 🚀 GPU مجاني من Google
- 💾 ذاكرة كبيرة (12GB GPU RAM)
- ⚡ تدريب سريع ومتقدم
- 🔄 تحديث تلقائي للتقدم

In [ ]:
# 📦 تثبيت المكتبات المطلوبة
!pip install torch torchvision torchaudio
!pip install transformers datasets tokenizers accelerate
!pip install pandas numpy tqdm matplotlib seaborn
!pip install google-colab-utils

# 🔍 فحص GPU
import torch
print(f"🚀 CUDA متاح: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"💻 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 ذاكرة GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 📂 رفع البيانات من Google Drive
from google.colab import drive
drive.mount('/content/drive')

# أو رفع ملف ZIP
from google.colab import files
print("📤 ارفع ملف البيانات (ZIP):")
uploaded = files.upload()

# استخراج البيانات
import zipfile
for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/training_data')
    print(f"✅ تم استخراج {filename}")

In [ ]:
# ⚙️ إعدادات التدريب
CONFIG = {
  "model_name": "microsoft/DialoGPT-medium",
  "epochs": 3,
  "batch_size": 8,
  "learning_rate": 5e-05,
  "max_length": 512
}

print("🔧 إعدادات التدريب:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# 🤖 نظام التدريب المتقدم
import json
import torch
import pandas as pd
from pathlib import Path
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import logging
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time

class ColabTrainer:
    def __init__(self, config):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"🚀 التدريب على: {self.device}")
    
    def load_data(self, data_path='/content/training_data'):
        """تحميل البيانات"""
        print("📊 تحميل البيانات...")
        
        data_path = Path(data_path)
        all_texts = []
        
        # قراءة ملفات JSON
        json_files = list(data_path.rglob('*.json'))
        print(f"📁 عثر على {len(json_files)} ملف JSON")
        
        for file_path in tqdm(json_files[:50], desc="معالجة الملفات"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                if isinstance(data, list):
                    for item in data:
                        text = self._extract_text(item)
                        if text and len(text) > 10:
                            all_texts.append(text)
                elif isinstance(data, dict):
                    text = self._extract_text(data)
                    if text and len(text) > 10:
                        all_texts.append(text)
                        
            except Exception as e:
                print(f"⚠️ خطأ في {file_path}: {e}")
        
        print(f"✅ تم تحميل {len(all_texts):,} نص")
        return Dataset.from_dict({'text': all_texts})
    
    def _extract_text(self, item):
        """استخراج النص"""
        if isinstance(item, str):
            return item
        
        text_fields = ['text', 'content', 'source', 'code', 'prompt', 'response']
        for field in text_fields:
            if isinstance(item, dict) and field in item:
                if isinstance(item[field], str):
                    return item[field]
                elif isinstance(item[field], list):
                    return ''.join(item[field])
        
        return str(item) if item else None
    
    def prepare_model_and_tokenizer(self):
        """تحضير النموذج والمُرمز"""
        print(f"🤖 تحميل النموذج: {self.config['model_name']}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.config['model_name'])
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.config['model_name'],
            torch_dtype=torch.float16,
            device_map='auto'
        )
        
        print(f"✅ تم تحميل النموذج على {next(self.model.parameters()).device}")
    
    def tokenize_dataset(self, dataset):
        """ترميز البيانات"""
        print("🔤 ترميز البيانات...")
        
        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                padding=True,
                max_length=self.config['max_length'],
                return_tensors='pt'
            )
        
        tokenized = dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=dataset.column_names
        )
        
        print(f"✅ تم ترميز {len(tokenized)} عينة")
        return tokenized
    
    def train_model(self, tokenized_dataset):
        """تدريب النموذج"""
        print("🔥 بدء التدريب...")
        
        training_args = TrainingArguments(
            output_dir='/content/fine_tuned_model',
            num_train_epochs=self.config['epochs'],
            per_device_train_batch_size=self.config['batch_size'],
            gradient_accumulation_steps=2,
            warmup_steps=500,
            learning_rate=self.config['learning_rate'],
            fp16=True,
            logging_steps=50,
            save_steps=1000,
            save_total_limit=2,
            dataloader_pin_memory=False,
            report_to=None
        )
        
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False
        )
        
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer,
        )
        
        # بدء التدريب مع مراقبة الوقت
        start_time = time.time()
        trainer.train()
        end_time = time.time()
        
        training_time = end_time - start_time
        print(f"⏱️ وقت التدريب: {training_time/60:.1f} دقيقة")
        
        # حفظ النموذج
        trainer.save_model('/content/final_model')
        self.tokenizer.save_pretrained('/content/final_model')
        
        print("💾 تم حفظ النموذج في /content/final_model")
        return trainer

# 🚀 تشغيل التدريب
trainer = ColabTrainer(CONFIG)
print("✅ تم إنشاء المدرب")

In [ ]:
# 🏃‍♂️ تشغيل التدريب الكامل
print("🔥 بدء التدريب الكامل...")

# 1. تحضير النموذج
trainer.prepare_model_and_tokenizer()

# 2. تحميل البيانات
dataset = trainer.load_data()

# 3. ترميز البيانات
tokenized_dataset = trainer.tokenize_dataset(dataset)

# 4. بدء التدريب
trained_model = trainer.train_model(tokenized_dataset)

print("🎉 انتهى التدريب بنجاح!")

In [ ]:
# 📥 تحميل النموذج المدرب
import zipfile
from google.colab import files

print("📦 ضغط النموذج المدرب...")

# ضغط النموذج
with zipfile.ZipFile('/content/fine_tuned_model.zip', 'w') as zipf:
    for root, dirs, files_list in os.walk('/content/final_model'):
        for file in files_list:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, '/content/final_model')
            zipf.write(file_path, arcname)

print("📤 تحميل النموذج...")
files.download('/content/fine_tuned_model.zip')

print("✅ تم تحميل النموذج بنجاح!")

In [ ]:
# 🧪 اختبار النموذج المدرب
print("🧪 اختبار النموذج المدرب...")

# تحميل النموذج للاختبار
from transformers import pipeline

generator = pipeline(
    'text-generation',
    model='/content/final_model',
    tokenizer='/content/final_model',
    device=0 if torch.cuda.is_available() else -1
)

# اختبارات سريعة
test_prompts = [
    "مرحباً، كيف يمكنني",
    "const myFunction = () => {",
    "import React from"
]

for prompt in test_prompts:
    print(f"\n🔤 المدخل: {prompt}")
    result = generator(prompt, max_length=100, num_return_sequences=1)
    print(f"🤖 المخرج: {result[0]['generated_text']}")
    print("-" * 50)